In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def conv_layer(x, num_filters=32):
    x = layers.Conv2D(num_filters, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    return x

inputs = keras.Input(shape=(32, 1024, 1))
x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = conv_layer(x, 32)
x = conv_layer(x, 32)
x = conv_layer(x, 64)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(4, activation='linear')(x)

model = keras.Model(inputs=inputs, 
                    outputs=outputs,
                    name='cnn_model')

model.compile(loss='mean_squared_error',
              optimizer='adam', metrics=['accuracy'])

keras.utils.plot_model(model, '2sig_simple_cnn_shapes.pdf', show_shapes=True)
model.summary()

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Activation, Flatten, Dropout, Conv2D, Conv3D, Input, Dense, BatchNormalization
from tensorflow.keras.models import Model

activation = 'relu'
def Residual(x, layers=32):
    conv = Conv2D(layers, (3, 3), padding='same')(x)
    residual =  keras.layers.add([x, conv])
    act = Activation(activation)(residual)
    normed = BatchNormalization()(act)
    return normed

inputs = Input(shape=(32, 1024, 2))
# 3x3 conv with 3 output channels (same as input channels)

strided0 = Conv2D(32, (3, 3), strides=2, activation=activation)(inputs)
# strided0 = Activation(activation)(strided0)

r1 = Residual(strided0, 32)
strided1 = Conv2D(32, (3, 3), strides=2, activation=activation)(r1)
# strided1 = Activation(activation)(strided1)

r2 = Residual(strided1, 32)
strided2 = Conv2D(64, (3, 3), strides=2, activation=activation)(r2)
# strided2 = Activation(activation)(strided2)


# a layer instance is callable on a tensor, and returns a tensor
flat = Flatten()(strided2)
dense0 = Dense(64, activation=activation)(flat)
d = Dense(2048, activation=activation)(dense0)
d = Dropout(0.5)(d)
predictions = Dense(4, activation='linear')(d)

model = Model(inputs=inputs, outputs=predictions)
model.compile(loss='mean_squared_error',
              optimizer='adam',)


from tensorflow.keras.utils import plot_model
plot_model(model, to_file='2sig_resnet_2chans_shapes.pdf', show_shapes=True)